In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv("/kaggle/input/vehicle-dataset-from-cardekho/car data.csv")
df.head()

In [ ]:
df_na=df.isna().sum()
df_null=df.isnull().sum()
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.columns
#creating derived feature
#df['no_of years']=2020-int(df['year'])
final_dataSet=df[[ 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]
final_dataSet.head()

In [ ]:
final_dataSet[['current_year']]=2020
final_dataSet[['num_of_years']]=final_dataSet['current_year']-final_dataSet['Year']
final_dataSet.head()

In [ ]:
final_dataSet.drop(['Year'],axis=1,inplace=True)
final_dataSet.drop(['current_year'],axis=1,inplace=True)

In [ ]:
#for onehot encoding categorical features
final_dataSet=pd.get_dummies(final_dataSet,drop_first=True)

In [ ]:
final_dataSet.head()

In [ ]:
#finding corelation
final_dataSet.corr()

In [ ]:
import seaborn as sns
sns.pairplot(final_dataSet)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
cormat=final_dataSet.corr()
tot_corr=cormat.index
plt.figure(figsize=(20,20))
g=sns.heatmap(final_dataSet[tot_corr].corr(),annot=True,cmap="CMRmap")

In [ ]:
# making dependent and independent feature
X= final_dataSet.iloc[:,1:]
Y=final_dataSet.iloc[:,0]


In [ ]:
X.head()


In [ ]:
Y.head()

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model=ExtraTreesRegressor()
model.fit(X,Y)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model_reg= RandomForestRegressor()



In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
print(n_estimators)

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
from sklearn.model_selection import RandomizedSearchCV
rg_cv = RandomizedSearchCV(estimator = model_reg, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rg_cv.fit(X_train,Y_train)

In [ ]:
pred=rg_cv.predict(X_test)

In [ ]:
pred

In [ ]:
sns.distplot(Y_test-pred)

In [ ]:
plt.scatter(Y_test,pred)